#Libraries and Data

In [1]:
#import libraries
import numpy as np
import pandas as pd
import itertools
from prophet import Prophet
import yfinance
import pickle
import xgboost as xgb
from datetime import datetime
from ta import add_all_ta_features

from dataProcessing import DataProcessing
# Remove Future Warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import StandardScaler




from sklearn.model_selection import train_test_split

# Machine Learning
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

# Evaluation
from sklearn.metrics import precision_score



/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
def load_and_setup_data(sybmol):
    df = pd.read_csv("../stock_historical_data/{}.csv".format(sybmol))
  #  df.set_index("Date", inplace=True)
   # new_data = pd.DataFrame(input_data)
   # new_data.set_index("Date", inplace=True)
    #concatenated_df = pd.concat([df,new_data])
    return df

def fetch_stock_data(tickers):
    data = yfinance.download(tickers,period='1d')
    return data
    

In [8]:
#symbols = ["CONCOR.NS","ELGIEQUIP.NS"]
symbols = ["OLECTRA.NS","CONCOR.NS","ELGIEQUIP.NS","IOC.NS","BEL.NS","TATAELXSI.NS","^NSEI","HDFCBANK.NS","TATAMOTORS.NS","SBIN.NS","TCS.NS","TITAN.NS","SUNPHARMA.BO","TECHM.NS", "ASIANPAINT.NS","TATACONSUM.NS"]

#symbols = ["^NSEI","BEL.NS"]

Icdate = 0
Iresult = 1
IisClassification = 2
Imodel = 3
Istock = 4
Iclose = 5
Iprob = 6
  # Add the tickers you want to fetch data for
loaded_models = {}
result = []

final_result = {}
final_target = {}
stock_data = fetch_stock_data(symbols)
#print(stock_data)
finaldata = {}
current_date = datetime.now().strftime("%d-%m-%Y")

for symbol in symbols:
    stock_name = symbol
    print(stock_name)
    if symbol == "^NSEI":
        stock_name = "NSEI"
 
    data = load_and_setup_data(symbol)
    data = data[["Open", "High", "Low", "Close","Volume"]]
    data['weekday'] =pd.to_datetime(data.index).dayofweek
   # data.to_csv(f"../stock_historical_data/{symbol}.csv")
    data = add_all_ta_features(data, open="Open", high="High", low="Low", close="Close", volume="Volume", fillna=True)
 
    # Find NaN Rows
    #na_list = data.columns[data.isna().any().tolist()]
   # data.drop(columns=na_list, inplace=True)
    # Handle inf values
    data.replace([np.inf, -np.inf], np.nan, inplace=True)
    data.dropna(axis=1, inplace=True)
    #data = data.drop(['volatility_kchi','volatility_kcli'], axis=1)

    #df_stationary.head()
   # print(data)
    non_stationaries = []
    for col in data.columns:
        if col != "volatility_kchi" and col != "volatility_kcli" and col != "weekday":
            #print(col)
            dftest = adfuller(data[col].values)
            p_value = dftest[1]
            t_test = dftest[0] < dftest[4]["1%"]
            if p_value > 0.05 or not t_test:
                non_stationaries.append(col)
    
    df_stationary = data.copy()
    
 
  
    loaded_model = {}
    feature_item = {}
    with open('../TrainedModel/indicator/{}_model_2.pkl'.format(symbol), 'rb') as f:
        loaded_model = pickle.load(f)
    with open('../TrainedModel/indicator/{}_features.txt'.format(symbol), 'rb') as f:
        feature_item = pickle.load(f)
   # print(feature_item,symbol,len(feature_item))
    #print("point zero :" ,len(df_stationary.columns))

    df_stationary[non_stationaries] = df_stationary[non_stationaries].pct_change()
    df_stationary = df_stationary.iloc[1:]
    # Find NaN Rows
    na_list = df_stationary.columns[df_stationary.isna().any().tolist()]
    
    df_stationary.drop(columns=na_list, inplace=True)
    #print(feature_item)
    #print("point one :" ,len(df_stationary.columns))
    df_stationary = df_stationary[feature_item]
    #print("point two :" ,len(df_stationary.columns))

#    print(df_stationary)
    df_stationary = df_stationary.iloc[1:]
    
    df_stationary[np.isinf(df_stationary)] = np.nan  # Replace inf with NaN
    df_stationary = df_stationary.dropna()
    #print("point three",len(df_stationary.columns),symbol)
    has_inf = "no inf data"
    if np.isinf(df_stationary).any().any():
        has_inf = "has inf data"
    df_stationary.replace(np.inf, np.nan, inplace=True)
    df_stationary.dropna(inplace=True)

        
    #print("check infifty value in",has_inf)
   # print(symbol," :",len(df_stationary.columns),len(feature_item))
    finaldata[symbol] = df_stationary 
    df_stationary = StandardScaler().fit_transform(df_stationary)
    y_pred = loaded_model.predict(df_stationary)
    y_pred = loaded_model.predict(df_stationary)

    #print("got result",symbol)
    # Specify Target

    #final training
    loaded_models[symbol] = loaded_model
    stock_name_only = stock_name.replace(".NS","")
   
    train_yhat = loaded_model.predict(df_stationary[-1:])
    train_yhat_proba = loaded_model.predict_proba(df_stationary[-1:])
    greater = 0
    one = train_yhat_proba[0][0]
    two = train_yhat_proba[0][1]
    if one > two:
        greater = one*100
    else:
        greater = one*100
    print(greater)
    final_result[symbol] = [current_date,train_yhat[0],1,'indicator',stock_name_only,stock_data.head(1).Close[symbol].values[0],round(greater,2)]
    result.append([current_date,train_yhat[0],1,'indicator',stock_name_only,stock_data.head(1).Close[symbol].values[0],round(greater,2)])

print(result)


[*********************100%***********************]  16 of 16 completed
OLECTRA.NS


/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in scalar divide
  dip[idx] = 100 * (self._dip[idx] / value)
/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in scalar divide
  din[idx] = 100 * (self._din[idx] / value)


50.550213708854585
CONCOR.NS


/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in scalar divide
  dip[idx] = 100 * (self._dip[idx] / value)
/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in scalar divide
  din[idx] = 100 * (self._din[idx] / value)


36.347914136726374
ELGIEQUIP.NS


...

81.14443304882514
ASIANPAINT.NS


/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in scalar divide
  dip[idx] = 100 * (self._dip[idx] / value)
/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in scalar divide
  din[idx] = 100 * (self._din[idx] / value)


23.830528145114602
TATACONSUM.NS


/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in scalar divide
  dip[idx] = 100 * (self._dip[idx] / value)
/Users/vipin/opt/anaconda3/envs/my_own/lib/python3.9/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in scalar divide
  din[idx] = 100 * (self._din[idx] / value)


35.63127202400083
[['01-08-2023', -1, 1, 'indicator', 'OLECTRA', 1158.449951171875, 50.55], ['01-08-2023', 1, 1, 'indicator', 'CONCOR', 693.5499877929688, 36.35], ['01-08-2023', -1, 1, 'indicator', 'ELGIEQUIP', 539.7000122070312, 59.3], ['01-08-2023', 1, 1, 'indicator', 'IOC', 94.0, 22.86], ['01-08-2023', -1, 1, 'indicator', 'BEL', 130.4499969482422, 53.09], ['01-08-2023', -1, 1, 'indicator', 'TATAELXSI', 7185.0498046875, 52.47], ['01-08-2023', 1, 1, 'indicator', 'NSEI', 19733.55078125, 43.86], ['01-08-2023', -1, 1, 'indicator', 'HDFCBANK', 1662.25, 62.77], ['01-08-2023', -1, 1, 'indicator', 'TATAMOTORS', 643.6500244140625, 50.81], ['01-08-2023', 1, 1, 'indicator', 'SBIN', 611.8499755859375, 31.78], ['01-08-2023', -1, 1, 'indicator', 'TCS', 3452.050048828125, 61.2], ['01-08-2023', -1, 1, 'indicator', 'TITAN', 3001.89990234375, 54.95], ['01-08-2023', -1, 1, 'indicator', 'SUNPHARMA.BO', 1143.6500244140625, 87.57], ['01-08-2023', -1, 1, 'indicator', 'TECHM', 1143.5, 81.14], ['01-08-2023',

In [9]:

dataa = { "data":[]} 

for res in result:
    dataa["data"].append({ 
            "cdate": res[Icdate],
            "result": str(res[Iresult]),
            "pre_close" : str(res[Iclose]),            
            "isClassification": res[IisClassification],
            "model": res[Imodel],
            "stock": res[Istock],
            "prob": str(res[Iprob]),
    })
    
dataa

{'data': [{'cdate': '01-08-2023',
   'result': '-1',
   'pre_close': '1158.449951171875',
   'isClassification': 1,
   'model': 'indicator',
   'stock': 'OLECTRA',
   'prob': '50.55'},
  {'cdate': '01-08-2023',
   'result': '1',
   'pre_close': '693.5499877929688',
   'isClassification': 1,
   'model': 'indicator',
   'stock': 'CONCOR',
   'prob': '36.35'},
  {'cdate': '01-08-2023',
   'result': '-1',
   'pre_close': '539.7000122070312',
   'isClassification': 1,
   'model': 'indicator',
   'stock': 'ELGIEQUIP',
   'prob': '59.3'},
  {'cdate': '01-08-2023',
   'result': '1',
   'pre_close': '94.0',
   'isClassification': 1,
   'model': 'indicator',
   'stock': 'IOC',
   'prob': '22.86'},
  {'cdate': '01-08-2023',
   'result': '-1',
   'pre_close': '130.4499969482422',
   'isClassification': 1,
   'model': 'indicator',
   'stock': 'BEL',
   'prob': '53.09'},
  {'cdate': '01-08-2023',
   'result': '-1',
   'pre_close': '7185.0498046875',
   'isClassification': 1,
   'model': 'indicator',

In [10]:
import requests
import json

url = "https://website-development-kerala.com/api_214124524/ai_model_daily_runner.php"

response = requests.post(url, json=dataa)
print(response.status_code)
print(response)

200
<Response [200]>


In [ ]:
#final_target

In [ ]:
for symbol in symbols:
  
    # Set Target (for Supervised ML later on)
    finaldata[symbol]["TARGET"] = -1
    finaldata[symbol].loc[finaldata[symbol]["Close"].shift(-1) > finaldata[symbol]["Close"], "TARGET"] = 1
    finaldata[symbol].dropna(inplace=True)
    #print(finaldata)
    df_tar = finaldata[symbol][["TARGET"]]
    df_stationar = StandardScaler().fit_transform(finaldata[symbol].iloc[:, :-1])
   # print(df_tar)

    retrained_model = loaded_models[symbol].fit(df_stationar,df_tar)
    print(df_stationar)
    with open('../TrainedModel/indicator/{}_model_2.pkl'.format(symbol), 'wb') as f:
        pickle.dump(retrained_model, f)
